In [ ]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

# part 1 - Tuning batch and epoch

In [ ]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [ ]:
#viewing data
data.head()

In [ ]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [ ]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [ ]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [ ]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# part 2- Optimizing Activation Function

In [ ]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [ ]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# How can you improve accuracy of the model? 

In [ ]:
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu', kernel_initializer='uniform'))
    mymodel.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [ ]:
model = KerasClassifier(build_fn=create_my_model, epochs=700, batch_size=50)

In [ ]:
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad', 'adam']
parameter_grid = dict(optimizer=optimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
#best acurccy i got : 0.756510 with running this code i use collab and work with me 